In [1]:
#Init
import numpy as np
import cPickle as pkl
import pandas as pd
import csv

path='/home/jorge/proyectos/tesis/RNN/sentiment/'


In [2]:
#Import sequences & dictionary generated in the notebook imdb_keras_preprocess_and_model
with open("imdb.pkl", "rb") as f:
    (X_train, y_train) = pkl.load(f)
    (X_test, y_test) = pkl.load(f)
print len(X_train), len(y_train)
print len(X_test), len(y_test)

imdb_dict = pkl.load( open( "imdb.dict.pkl", "rb" ) )
print len(imdb_dict)

25000 25000
25000 25000
101758


In [3]:
#Average length of sentences
print np.mean([len(x) for x in X_train])

284.549


In [4]:
#Import wikipedia embeddings
#Downloaded from http://nlp.stanford.edu/projects/glove/
embed_dim=300

df_glove = pd.read_csv("/home/jorge/data/text/glove/glove.6B.300d.txt", index_col=0 ,sep=' ',
                   header = None, quoting=csv.QUOTE_NONE, encoding='utf-8')

#Merge with the dictionary of the current texts
#Inner join, only words in the corpus and in glove.
df_glove = df_glove.merge(pd.DataFrame.from_dict(imdb_dict, orient='index'), left_index=True, right_index=True)
print df_glove.shape


#Create dictionary: word_number_id --> [glove vector associated]
glove={}
for i,r in df_glove[:].iterrows():
    glove[int(r[0])] = [r[j] for j in range(1,embed_dim+1)]
    
print len(glove)

(64083, 301)
64083


/usr/local/lib/python2.7/dist-packages/pandas-0.17.0-py2.7-linux-x86_64.egg/pandas/core/index.py:1840: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  indexer = self._engine.get_indexer(target._values)


In [5]:
#First version:
# - Sequences of fixed length
maxlen = 300  # cut texts after this number of words (among top max_features most common words)

#Max features: No relevant here, the filter is the cross with the globe embedings
'''
max_features = 20000
def remove_features(x):
    return [[0 if w >= max_features else w for w in sen] for sen in x]

print np.max(X_train)
print np.max(X_test)
X_train = remove_features(X_train)
X_test  = remove_features(X_test)
print np.max(X_train)
print np.max(X_test)
'''

'\nmax_features = 20000\ndef remove_features(x):\n    return [[0 if w >= max_features else w for w in sen] for sen in x]\n\nprint np.max(X_train)\nprint np.max(X_test)\nX_train = remove_features(X_train)\nX_test  = remove_features(X_test)\nprint np.max(X_train)\nprint np.max(X_test)\n'

In [6]:
#Create embeddings 3D tensors
# - Complete with zeros at the begining. (better at the end?)

def embedd(x):
    r = np.zeros((maxlen,embed_dim))
    pos = maxlen-1
    for i in xrange(len(x),0,-1):
        finded = True
        try:
            v = np.array([glove[x[i-1]]])
        except:
            finded = False
        if finded and pos>=0:
            r[pos,:] = v 
            pos += -1
    return r
        
X_train_emb = np.array([embedd(s) for s in X_train])

print X_train_emb.shape

(25000, 300, 300)


In [7]:
X_test_emb = np.array([embedd(s) for s in X_test])

In [8]:
#Define the model

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape=(maxlen, embed_dim)))  # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              class_mode="binary")


Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled)


In [9]:
batch_size = 128

print("Train...")
model.fit(X_train_emb, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test_emb, y_test), show_accuracy=True)


Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 226s - loss: 0.5616 - acc: 0.7080 - val_loss: 0.5468 - val_acc: 0.7286
Epoch 2/15
25000/25000 [==============================] - 209s - loss: 0.4460 - acc: 0.8008 - val_loss: 0.3766 - val_acc: 0.8385
Epoch 3/15
25000/25000 [==============================] - 209s - loss: 0.3865 - acc: 0.8378 - val_loss: 0.3445 - val_acc: 0.8540
Epoch 4/15
25000/25000 [==============================] - 209s - loss: 0.3369 - acc: 0.8595 - val_loss: 0.3394 - val_acc: 0.8552
Epoch 5/15
25000/25000 [==============================] - 208s - loss: 0.3100 - acc: 0.8730 - val_loss: 0.2947 - val_acc: 0.8765
Epoch 6/15
25000/25000 [==============================] - 209s - loss: 0.2806 - acc: 0.8861 - val_loss: 0.2867 - val_acc: 0.8782
Epoch 7/15
25000/25000 [==============================] - 209s - loss: 0.2565 - acc: 0.8980 - val_loss: 0.2718 - val_acc: 0.8909
Epoch 8/15
25000/25000 [==============

In [10]:
score, acc = model.evaluate(X_test_emb, y_test,
                            batch_size=batch_size,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, model.predict_proba(X_test_emb))


25000/25000 [==============================] - 71s    
('Test score:', 0.37100702987670897)
('Test accuracy:', 0.89039999999999997)
25000/25000 [==============================] - 45s    
0.9538336608


In [20]:
#Save model
import cPickle as pkl
import sys
sys.setrecursionlimit(10000)

pkl.dump( model, open('/home/jorge/proyectos/tesis/RNN/sentiment/glove_lstm128_imdb_model_keras_model.pkl', "wb" ) )


In [9]:
#Second model, two LSTM layers

model2 = Sequential()
model2.add(LSTM(512, return_sequences=False, input_shape=(maxlen, embed_dim)))
model2.add(Dropout(0.5))
model2.add(Dense(1))
model2.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model2.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              class_mode="binary")


In [10]:
batch_size = 256

print("Train...")
model2.fit(X_train_emb, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test_emb, y_test), show_accuracy=True)


Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 126s - loss: 0.6777 - acc: 0.5933 - val_loss: 0.6149 - val_acc: 0.6782
Epoch 2/15
25000/25000 [==============================] - 134s - loss: 0.6306 - acc: 0.6590 - val_loss: 0.5700 - val_acc: 0.7150
Epoch 3/15
25000/25000 [==============================] - 133s - loss: 0.6013 - acc: 0.6858 - val_loss: 0.5548 - val_acc: 0.7344
Epoch 4/15
25000/25000 [==============================] - 134s - loss: 0.5434 - acc: 0.7318 - val_loss: 0.5270 - val_acc: 0.7566
Epoch 5/15
25000/25000 [==============================] - 134s - loss: 0.5154 - acc: 0.7589 - val_loss: 0.5142 - val_acc: 0.7519
Epoch 6/15
25000/25000 [==============================] - 134s - loss: 0.4780 - acc: 0.7785 - val_loss: 0.4552 - val_acc: 0.7948
Epoch 7/15
25000/25000 [==============================] - 134s - loss: 0.4521 - acc: 0.7962 - val_loss: 0.6143 - val_acc: 0.6431
Epoch 8/15
25000/25000 [==============

In [12]:
score, acc = model2.evaluate(X_test_emb, y_test,
                            batch_size=batch_size,
                            show_accuracy=True)
print('Test score:', score)
print('Test accuracy:', acc)

from sklearn.metrics import roc_auc_score
print roc_auc_score(y_test, model2.predict_proba(X_test_emb))


25000/25000 [==============================] - 34s    
('Test score:', 0.29102582094192503)
('Test accuracy:', 0.877)
25000/25000 [==============================] - 40s    
0.9491958336



Comparation of accuracy whit several models in http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf 

    Features                          Our Dataset 
    Bag of Words (bnc)                87.80 
    Bag of Words (b∆t’c)              88.23
    LDA                               67.425
    LSA                               83.96
    Our Semantic Only                 87.30
    Our Full                          87.44
    Our Full, Additional Unlabeled    87.99
    Our Semantic + Bag of Words (bnc) 88.28
    Our Full + Bag of Words (bnc)     88.33
    Our Full, Add’l Unlabeled + Bag of Words (bnc) 88.89